In [1]:
import math
import numpy as np

class DenseLayer:
    def __init__(self, units, activation, input_dim, init, use_bias=False):
        self.units = units
        self.input_dim = input_dim
        
        if activation == 'sigmoid':
            self.activation_function = self.sigmoid
        else:
            print('Activation function not supported')
        
        if init == 'uniform':
            self.weight_matrix = np.random.uniform(0.0, 1.0, size=(self.units, input_dim)) 
        elif init == 'random':
            self.weight_matrix = np.random.random(size=(self.units, input_dim))
        else:
            print('Init function not supported')
        
        self.delta_weight_matrix_before = np.zeros((self.units, input_dim))
        
        self.use_bias = use_bias
        if self.use_bias:
            bias = np.zeros((units, 1))
            self.weight_matrix = np.hstack((self.weight_matrix, bias))
            self.delta_weight_matrix_before = np.hstack((self.delta_weight_matrix_before, np.zeros((units, 1))))
            
    def calculate_sigma(self, input_list):
        if self.use_bias:
            input_list = np.append(input_list, 1)
        
        result_list = np.array([])
        for weight_neuron in self.weight_matrix:
            result_list = np.append(result_list, np.dot(weight_neuron, input_list))
        return np.array(result_list)
    
    def calculate_output(self, input_list):
        output_list = np.array([])
        for sigma_neuron in self.calculate_sigma(input_list):
            output_list = np.append(output_list, self.activation_function(sigma_neuron))
        return output_list
    
    def calculate_error_output_layer(self, output_list, target_list):
        """
        Use this if the layer is output layer
        """
        result_list = np.array([])
        for index, output in enumerate(output_list):
            error_output = output * (1 - output) * (target_list[index] - output)
            result_list = np.append(result_list, error_output)    
        return result_list
    
    def calculate_error_hidden_layer(self, output_list, error_output_list, output_layer_weight_matrix):
        """
        Use this if the layer is hidden layer
        """
        result_list = np.array([])
        for index, output in enumerate(output_list):
            sigma_output_error = 0
            for unit_number, error_output in enumerate(error_output_list):
                sigma_output_error += output_layer_weight_matrix[unit_number][index] * error_output
            error_hidden = output * (1 - output) * sigma_output_error
            result_list = np.append(result_list, error_hidden)
        return result_list
    
    def update_weight(self, lr, layer_error_list, input_list, momentum=None):
        """
        Function to update weight
        """
        if momentum == None:
            for j, unit in enumerate(self.weight_matrix): #j  
                for i, source in enumerate(unit): #i
                    delta_weight = lr * layer_error_list[j] * input_list[i]
                    new_weight = source + delta_weight
                    self.weight_matrix[j][i] = new_weight
        else:
            for j, unit in enumerate(self.weight_matrix): #j  
                for i, source in enumerate(unit): #i
                    delta_weight = lr * layer_error_list[j] * input_list[i] + momentum * self.delta_weight_matrix_before[j][i]
                    self.delta_weight_matrix_before[j][i] = delta_weight
                    
                    new_weight = source + delta_weight
                    self.weight_matrix[j][i] = new_weight
                    
            
    def sigmoid(self, x):
        return 1 / (1 + math.exp(-x))

In [2]:
np.random.seed(5)
layer = DenseLayer(3, 'sigmoid', 2, 'uniform', True)
layer.calculate_sigma([3, 2])

array([2.40744413, 2.45737928, 2.68872129])

In [3]:
layer.calculate_output([3, 2])

array([0.9173932 , 0.92109941, 0.93635782])

# Test Feedforward

In [4]:
layer_hidden = DenseLayer(2, 'sigmoid', 2, 'uniform', True)
layer_hidden.weight_matrix = np.array([[-0.2, 0.1, 0.1], [-0.1, 0.3, 0.1]])

layer_hidden.calculate_sigma([0.1, 0.9])

array([0.17, 0.36])

# Test Calculate Hidden Ouput

In [5]:
layer_hidden_output = DenseLayer(2, 'sigmoid', 2, 'uniform', True)
layer_hidden_output.calculate_error_hidden_layer([0.542, 0.589], [0.0663], [[0.2, 0.3]])

array([0.00329161, 0.00481495])

# Test Calculate Error Output

In [6]:
layer_error_output = DenseLayer(1, 'sigmoid', 2, 'uniform', True)
layer_error_output.calculate_error_output_layer([0.619], [0.9])

array([0.06627076])

# Test Update Weight Momentum 1

In [7]:
layer_test_update_weight = DenseLayer(1, 'sigmoid', 2, 'uniform', True)
layer_test_update_weight.weight_matrix = np.array([[0.2, 0.3, 0.2]])

layer_test_update_weight.delta_weight_matrix_before = np.array([[0.2, 0.3, 0.2]])
print(layer_test_update_weight.weight_matrix)
print(layer_test_update_weight.delta_weight_matrix_before)

layer_test_update_weight.update_weight(0.25, [0.0663], [0.542, 0.589, 1], 0.0001)
print(layer_test_update_weight.delta_weight_matrix_before)
print(layer_test_update_weight.weight_matrix)

[[0.2 0.3 0.2]]
[[0.2 0.3 0.2]]
[[0.00900365 0.00979267 0.016595  ]]
[[0.20900365 0.30979267 0.216595  ]]


# Test Update Weight Momentum 2

In [8]:
layer_test_update_weight2 = DenseLayer(2, 'sigmoid', 2, 'uniform', True)
layer_test_update_weight2.weight_matrix = np.array([[-0.2, 0.1, 0.1], [-0.1, 0.3, 0.1]])

layer_test_update_weight2.delta_weight_matrix_before = np.array([[-0.2, 0.1, 0.1], [-0.1, 0.3, 0.1]])
print(layer_test_update_weight2.weight_matrix)
print(layer_test_update_weight2.delta_weight_matrix_before)

layer_test_update_weight2.update_weight(0.25, [0.0033, 0.0049], [0.1, 0.9, 1], 0.0001)
print(layer_test_update_weight2.delta_weight_matrix_before)
print(layer_test_update_weight2.weight_matrix)

[[-0.2  0.1  0.1]
 [-0.1  0.3  0.1]]
[[-0.2  0.1  0.1]
 [-0.1  0.3  0.1]]
[[6.2500e-05 7.5250e-04 8.3500e-04]
 [1.1250e-04 1.1325e-03 1.2350e-03]]
[[-0.1999375  0.1007525  0.100835 ]
 [-0.0998875  0.3011325  0.101235 ]]


# Test Update Weight Without Momentum TODO

# Model Class

In [9]:
class LunakModel:
    def __init__(self):
        return